# XGBoost `XGBRegressor` — Hiperparâmetros no **construtor** vs **treino** (fit)

> Autor: Lucas Bussinger — Gerado pelo M365 Copilot em 2026-01-27 12:14
> Objetivo: Demonstrar, com exemplos executáveis, **onde** cada hiperparâmetro do `XGBRegressor` é definido: no **modelo** (construtor) ou no **treino** (`fit`).


## Sumário
1. [Visão geral rápida](#visao-geral)
2. [Parâmetros no **construtor** (`XGBRegressor(...)`)](#construtor)
3. [Parâmetros no **treino** (`fit(...)`)](#treino)
4. [Exemplo completo com validação e *early stopping*](#exemplo-completo)
5. [Presets: GBTREE, DART e GBLINEAR](#presets)
6. [Inspecionando todos os parâmetros do estimador](#inspecao)


## 1) Visão geral rápida <a id="visao-geral"></a>
- **Construtor**: define *hiperparâmetros do modelo/booster* (e.g., `n_estimators`, `learning_rate`, `max_depth`, `subsample`, `colsample_*`, `reg_alpha`, `reg_lambda`, `tree_method`, `gamma`, `booster`, `objective`, etc.).
- **Treino (`fit`)**: define *parâmetros de avaliação e rotina de treinamento* (e.g., `eval_set`, `eval_metric`, `early_stopping_rounds`, `verbose`, `callbacks`, `sample_weight`, `sample_weight_eval_set`).


## 2) Parâmetros no **construtor** (`XGBRegressor(...)`) <a id="construtor"></a>
São passados na criação do estimador e controlam o **booster** e a **regularização**.


In [ ]:
from xgboost import XGBRegressor

# ------------------------------
# EXEMPLO: booster = "gbtree" (mais comum)
# ------------------------------
gbtree = XGBRegressor(
    # Geral
    booster="gbtree",          # 'gbtree' | 'dart' | 'gblinear'
    n_estimators=500,              # int >= 1
    learning_rate=0.05,            # (0, 1], típico 0.01–0.3 (alias de 'eta')
    random_state=42,               # semente

    # Complexidade/estrutura
    max_depth=8,                   # int >= 0
    min_child_weight=1.0,          # float >= 0
    max_delta_step=0.0,            # float >= 0
    max_leaves=0,                  # int >= 0 (0 desativa, útil com 'lossguide')
    grow_policy="depthwise",   # 'depthwise' | 'lossguide'
    max_bin=256,                   # int >= 2 (para 'hist'/'gpu_hist')

    # Amostragem (regularização estocástica)
    subsample=0.8,                 # (0, 1]
    colsample_bytree=0.8,          # (0, 1]
    colsample_bylevel=1.0,         # (0, 1]
    colsample_bynode=1.0,          # (0, 1]

    # Regularização
    reg_lambda=1.0,                # L2 >= 0
    reg_alpha=0.0,                 # L1 >= 0

    # Método de construção da árvore
    tree_method="hist",        # 'auto' | 'exact' | 'approx' | 'hist' | 'gpu_hist'

    # Diversos
    gamma=0.0,                     # >= 0, ganho mínimo para split
    scale_pos_weight=1.0,          # >= 0 (mais útil em classificação)
    sampling_method="uniform", # 'uniform' | 'gradient_based' (para hist)
    base_score=0.5,                # valor inicial de predição
    verbosity=1,                   # 0,1,2,3
    importance_type="gain",    # 'gain' | 'weight' | 'cover' | 'total_gain' | 'total_cover'

    # Objetivo (tarefa de regressão)
    objective="reg:squarederror",
    enable_categorical=False
)
gbtree

### Booster DART (parâmetros adicionais)
Além dos parâmetros de árvores, o DART adiciona *dropout* de árvores.


In [ ]:
dart = XGBRegressor(
    booster="dart",
    n_estimators=400,
    learning_rate=0.05,
    # Todos os params de gbtree acima tbm se aplicam aqui (árvore por baixo)
    max_depth=6,
    subsample=0.9,
    colsample_bytree=0.9,
    # Específicos do DART
    rate_drop=0.1,             # 0–1
    skip_drop=0.0,             # 0–1
    one_drop=False,            # bool
    max_drop=50,               # int >= 0
    sample_type="uniform", # 'uniform' | 'weighted'
    normalize_type="tree"  # 'tree' | 'forest'
)
dart

### Booster GBLINEAR (modelo linear)
Ignora hiperparâmetros de árvore e usa principalmente regularização.


In [ ]:

blinear = XGBRegressor(
    booster="gblinear",
    n_estimators=200,      # iterações do otimizador
    learning_rate=0.1,     # também usado no otimizador linear

    # Regularização
    reg_lambda=1.0,        # L2
    reg_alpha=0.0,         # L1

    # Algumas versões/compilações expõem:
    # lambda_bias=0.0,     # L2 no viés (se disponível)
    # feature_selector="cyclic",  # "cyclic" | "thrifty" | "greedy" (contexto interno)
    # top_k=0              # int >= 0
)


## 3) Parâmetros no **treino** (`fit(...)`) <a id="treino"></a>
Usados para **avaliação**, **validação**, *early stopping* e **pesos de amostra**.


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
import numpy as np

# Dados sintéticos para exemplo
X, y = make_regression(n_samples=5000, n_features=30, noise=15.0, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = gbtree

sample_weight = np.ones(len(X_train), dtype=float)
sample_weight_val = np.ones(len(X_val), dtype=float)

model.fit(
    X_train, y_train,
    # Avaliação/validação
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_metric=["rmse", "mae"],  # métrica de AVALIAÇÃO

    # Early stopping (requer eval_set)
    early_stopping_rounds=50,
    verbose=True,

    # Pesos
    sample_weight=sample_weight,
    sample_weight_eval_set=[sample_weight, sample_weight_val],
)

best_iter = getattr(model, 'best_iteration', None)
best_score = getattr(model, 'best_score', None)
best_iter, best_score


## 3.2  parametros no treino(fit)

In [ ]:

from sklearn.model_selection import train_test_split
import numpy as np

# Suponha X, y já preparados (numpy arrays ou pandas)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = gbtree  # de cima

# Exemplo de pesos por amostra (opcional)
sample_weight = np.ones(len(X_train), dtype=float)
sample_weight_val = np.ones(len(X_val), dtype=float)

model.fit(
    X_train, y_train,

    # ---------------------------
    # Avaliação/validação
    # ---------------------------
    eval_set=[(X_train, y_train), (X_val, y_val)],  # lista de tuplas
    eval_metric=[                                   # string ou lista de strings
        "rmse",     # comuns em regressão: "rmse", "mae", "mape"
        # Para objetivos específicos:
        # "poisson-nloglik", "gamma-nloglik", "aft-nloglik", etc.
    ],

    # ---------------------------
    # Early stopping (para parar nos melhores rounds)
    # ---------------------------
    early_stopping_rounds=50,   # int >= 1 (requer eval_set)
    verbose=True,               # imprime métricas a cada iteração

    # ---------------------------
    # Pesos e amostragem por instância
    # ---------------------------
    sample_weight=sample_weight,                 # (opcional)
    sample_weight_eval_set=[sample_weight, sample_weight_val],  # (opcional)

    # ---------------------------
    # Callbacks (avançado)
    # ---------------------------
    callbacks=None,             # pode passar lista de callbacks xgboost.callback.*
    # xgb_model=None,          # caminho/objeto para continuar treinamento (warm start)
)


## 4) Exemplo completo com validação e *early stopping* <a id="exemplo-completo"></a>


In [ ]:
from xgboost import XGBRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

X, y = make_regression(n_samples=3000, n_features=25, noise=10.0, random_state=0)
X_tr, X_va, y_tr, y_va = train_test_split(X, y, test_size=0.2, random_state=0)

model_full = XGBRegressor(
    booster="gbtree",
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=7,
    min_child_weight=1.0,
    subsample=0.85,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    reg_alpha=0.0,
    gamma=0.0,
    tree_method="hist",
    objective="reg:squarederror",
    random_state=42,
    verbosity=1
)

model_full.fit(
    X_tr, y_tr,
    eval_set=[(X_tr, y_tr), (X_va, y_va)],
    eval_metric=["rmse", "mae"],
    early_stopping_rounds=100,
    verbose=False
)
print("Melhor iteração:", getattr(model_full, 'best_iteration', None))
print("Melhor pontuação (última métrica):", getattr(model_full, 'best_score', None))


## 5) Presets: GBTREE, DART e GBLINEAR <a id="presets"></a>
Blocos prontos para reutilização em diferentes cenários.


In [ ]:
def make_gbtree(random_state=42):
    return XGBRegressor(
        booster="gbtree", n_estimators=1000, learning_rate=0.03, max_depth=7,
        subsample=0.85, colsample_bytree=0.9, reg_lambda=1.0, reg_alpha=0.0,
        tree_method="hist", objective="reg:squarederror", random_state=random_state
    )

def make_dart(random_state=42):
    return XGBRegressor(
        booster="dart", n_estimators=800, learning_rate=0.05, max_depth=6,
        subsample=0.9, colsample_bytree=0.9, rate_drop=0.1, skip_drop=0.0,
        sample_type="uniform", normalize_type="tree", objective="reg:squarederror",
        tree_method="hist", random_state=random_state
    )

def make_gblinear(random_state=42):
    return XGBRegressor(
        booster="gblinear", n_estimators=400, learning_rate=0.1,
        reg_lambda=1.0, reg_alpha=0.0, objective="reg:squarederror",
        random_state=random_state
    )

gbtree_preset = make_gbtree()
dart_preset = make_dart()
gblinear_preset = make_gblinear()
gbtree_preset, dart_preset, gblinear_preset


## 6) Inspecionando todos os parâmetros do estimador <a id="inspecao"></a>
Útil para descobrir tudo que o wrapper sklearn expõe na sua instalação.


In [ ]:
# A saída pode variar conforme a versão do XGBoost instalada.
XGBRegressor().get_params().keys()


---
### Notas
- `eval_metric` é parâmetro de **treino** (como avaliar), enquanto `objective` é do **modelo** (qual perda otimizar).
- `early_stopping_rounds` só funciona quando `eval_set` é fornecido.
- `n_estimators` define o **máximo** de boosting rounds; com *early stopping*, o número efetivo pode ser menor (`best_iteration`).
- Métricas comuns na regressão: `rmse`, `mae`, `mape`; para contagem: `poisson-nloglik`; para quantis (quando suportado): `quantile`.
- `tree_method="gpu_hist"` requer XGBoost com suporte a GPU.
